In [115]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [116]:
# %timeit sirve para evaluar el tiempo de ejecucion
event = pd.read_csv('events_up_to_01062018.csv')
label_train = pd.read_csv('labels_training_set.csv')
test = pd.read_csv('trocafone_kaggle_test.csv')

#projects_2017.head()trocafone_kaggle_test
#projects_2017

In [117]:
test_con_feature  = pd.read_csv('feature_test_paso_uno.csv')
train_con_feature = pd.read_csv('feature_training_paso_uno.csv')

In [118]:
len(train_con_feature)

19414

In [119]:
events_test = event.merge(test,on='person',how='inner')
events_test.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:30:30,viewed product,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,nan,NaN,NaN,NaN,NaN,NaN,...,Google,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:56,checkout,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:35,viewed product,4886f805,NaN,"9,287.00",Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
channel=events_training.groupby(['person','channel']).agg({'channel':'count'}).unstack()
channel.columns=['Direct','Email','Organic','Paid','Referral','Social','desconocido']
channel=channel.reset_index()
channel=channel.fillna(0)
channel.head()

,person,Direct,Email,Organic,Paid,Referral,Social,desconocido
0,0008ed71,0.00,0.00,0.00,0.00,2.00,0.00,0.00
1,000c79fe,0.00,0.00,0.00,1.00,0.00,0.00,0.00
2,001802e4,0.00,0.00,0.00,1.00,0.00,0.00,0.00
3,0019e639,0.00,0.00,2.00,16.00,1.00,0.00,0.00
4,001ca5ee,2.00,0.00,2.00,5.00,6.00,0.00,0.00


In [121]:
events_training['date'] = pd.to_datetime(events_training['timestamp'])

In [122]:
storage=events_training.groupby(['person' ,'storage']).agg({'storage':'count'}).unstack().fillna(0)  
storage.head() 

storage                                            \
storage                 128GB                 16GB                256GB   
person                                                                    
0008ed71                 0.00                 0.00                 0.00   
000c79fe                 4.00                 0.00                 0.00   
001802e4                 0.00                 0.00                 2.00   
0019e639                 0.00                98.00                 0.00   
001b0bf9                 2.00                 0.00                 0.00   

                                                                         \
storage                  32GB                  4GB                512MB   
person                                                                    
0008ed71                 2.00                 0.00                 0.00   
000c79fe                 0.00                 0.00                 0.00   
001802e4                 0.00                 0.00                 0.00   
0019e639                14.00                 0.00                 0.00   
001b0bf9                 0.00                 0.00                 0.00   

                                                    
storage                  64GB                  8GB  
person                                              
0008ed71                 1.00                 0.00  
000c79fe                 0.00                 0.00  
001802e4                 3.00                 0.00  
0019e639                 0.00                94.00  
001b0bf9                 3.00                 0.00

In [131]:
#Este metodo lo implementamos para poder encontrar la marca de cada producto
#La funcíon del mismo es recibir un objeto (que en este caso representa el producto 
#sobre el que actua el evento)
#y nos va a devolver la marca del producto

def hallarMarca(x):
   x=str(x)
   y=""
   if x.find('iPhone')==0:
       y="iPhone"
   elif x.find('Samsung')==0:
       y="Samsung"
   elif x.find('Motorola')==0:
       y="Motorola"
   elif x.find('iPad')==0:
       y="iPad"
   elif x.find('LG')==0:
       y="LG"
   elif x.find('Quantum')==0:
       y="Quantum"
   elif x.find('Blackberry')==0:
       y="Blackberry"
   elif x.find('Asus')==0:
       y="Asus"
   elif x.find('Lenovo')==0:
       y="Lenovo"
   elif x.find('Sony')==0:
       y="Sony"
   return y


In [134]:
#Aplico la funcíon hallarMarca a cada elemento de la columna 'models'
#El resultado es una serie con la marca de cada producto (en orden)

sMarcas=events_training.apply({'model':hallarMarca})
#Creo un nuevo dataframe con la columna 'mark', que representa las marcas de cada producto

dataframeConMarcas=events_training
dataframeConMarcas['mark']=sMarcas
dataframeConMarcas.head()
# grafico=conversiones['mark'].value_counts().plot(kind='barh',figsize=(10,6),title='Marca \

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label,date,mark
0,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.00,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018-05-18 00:11:27,iPhone
1,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.00,iPhone 5s,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018-05-18 00:23:33,iPhone
2,2018-05-18 00:16:10,viewed product,ad93850f,NaN,"5,907.00",iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018-05-18 00:16:10,iPhone
3,2018-05-18 00:14:55,viewed product,ad93850f,NaN,"6,023.00",iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018-05-18 00:14:55,iPhone
4,2018-05-18 00:11:26,ad campaign hit,ad93850f,/comprar/iphone/iphone-5s,nan,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018-05-18 00:11:26,


In [139]:
events_training['event'].value_counts()
.groupby(['person','mark']).agg({'mark':'count'})

viewed product       629233
brand listing        105993
visited site         102299
ad campaign hit       94998
generic listing       80453
searched products     63887
search engine hit     52829
checkout              32539
staticpage             5660
conversion             3564
lead                    431
Name: event, dtype: int64

In [180]:
a=dataframeConMarcas['event']=='viewed product'
b=dataframeConMarcas['event']=='checkout'
c=dataframeConMarcas['event']=='conversion'
data_mark=dataframeConMarcas[a|b|c].groupby(['person','mark']).agg({'mark':'count'}).unstack().fillna(0)
data_mark.columns=['desconocido','Asus','LG','Lenovo','Motorola','Quantum','Samsung','Sony','iPad','iPhone']
data_mark=data_mark.reset_index()
data_mark.head()


,person,desconocido,Asus,LG,Lenovo,Motorola,Quantum,Samsung,Sony,iPad,iPhone
0,0008ed71,3.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
1,000c79fe,13.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00
2,001802e4,14.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00
3,0019e639,265.00,0.00,1.00,0.00,138.00,0.00,64.00,0.00,0.00,3.00
4,001b0bf9,2.00,0.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,0.00


In [123]:
datatrain=pd.merge(train_con_feature,storage,on='person',how='left')
datatrain=datatrain.fillna(0)
datatrain.isnull().sum()

person                       0
ad campaign hit              0
brand listing                0
checkout                     0
conversion                   0
generic listing              0
lead                         0
search engine hit            0
searched products            0
staticpage                   0
viewed product               0
visited site                 0
mean_per_event               0
most_frequent_event          0
count_most_frequent_event    0
less_frequent_event          0
count_less_frequent_event    0
label                        0
(storage, 128GB)             0
(storage, 16GB)              0
(storage, 256GB)             0
(storage, 32GB)              0
(storage, 4GB)               0
(storage, 512MB)             0
(storage, 64GB)              0
(storage, 8GB)               0
dtype: int64

In [124]:
storage=events_training.groupby(['person' ,'storage']).agg({'storage':'count'})  
print (len(events_training))

1171886


In [191]:
def agregarcolumnas(archivo,archivo_con_feature):
    #archivo_con_feature es un archivo test o train con feature antes calculada
    #archivo es el train con label o el test
    
    
    #agrego divicion entre convert y checkout
    
    #cantidad_eventos=archivo.groupby(['person','event']).agg({'event':'count'})\
    #.unstack().fillna(0)
    #cantidad_eventos.columns=['ad campaign hit','brand listing','checkout','conversion','generic listing','lead','search engine hit','searched products','staticpage','viewed product','visited site']
    #cantidad_eventos=cantidad_eventos.reset_index()
    #cantidad_eventos['div_conver_check']=cantidad_eventos['conversion'].astype(float)/\
    #cantidad_eventos['checkout'].astype(float)
    #cantidad_eventos=cantidad_eventos.fillna(0)
    #cantidad_eventos=cantidad_eventos[['person','div_conver_check']]
    #cantidad_eventos.head()
    
    #agrego storage
    print (len(archivo))
    storage=archivo.groupby(['person' ,'storage']).agg({'storage':'count'}).unstack().fillna(0) 
    print (len(storage))
    storage.columns=['128GB','16GB','256GB','32GB','4GB','512MB','64GB','8GB']
    print (len(storage))
    storage=storage.reset_index() 
    print (len(storage))
    storage.head(5)
    print (len(storage))
    archivo_con_storage=pd.merge(archivo_con_feature,storage,on='person',how='left')
    archivo_con_storage=archivo_con_storage.fillna(0)
    print (len(archivo_con_storage))
    #agrego condition
    
    condition=archivo.groupby(['person' ,'condition']).agg({'condition':'count'}).unstack().fillna(0) 
  
    condition.columns=['Bom','Bom - Sem Touch ID','Excelente','Muito Bom','Novo']
    condition=condition.reset_index() 
    condition.head(5)
    archivo_con_storage_condition=pd.merge(archivo_con_storage,condition,on='person',how='left')
    archivo_con_storage_condition=archivo_con_storage_condition.fillna(0)
    print (len(archivo_con_storage_condition))
    
    #agrego columnas modelo
    
    #Aplico la funcíon hallarMarca a cada elemento de la columna 'models'
    #El resultado es una serie con la marca de cada producto (en orden)

    sMarcas=archivo.apply({'model':hallarMarca})
    #Creo un nuevo dataframe con la columna 'mark', que representa las marcas de cada producto

    dataframeConMarcas=archivo
    dataframeConMarcas['mark']=sMarcas
    dataframeConMarcas.head()
    a=dataframeConMarcas['event']=='viewed product'
    b=dataframeConMarcas['event']=='checkout'
    c=dataframeConMarcas['event']=='conversion'
    data_mark=dataframeConMarcas.groupby(['person','mark']).agg({'mark':'count'}).unstack().fillna(0)
    data_mark.columns=['desconocido_modelo','Asus','LG','Lenovo','Motorola','Quantum','Samsung','Sony','iPad','iPhone']
    data_mark=data_mark.reset_index()
    archivo_con_storage_condition_mark=pd.merge(archivo_con_storage_condition,data_mark,on='person',how='left')
    archivo_con_storage_condition_mark=archivo_con_storage_condition_mark.fillna(0)
    
    #agrego las columnas channel
    
    channel=archivo.groupby(['person','channel']).agg({'channel':'count'}).unstack()
    channel.columns=['Direct','Email','Organic','Paid','Referral','Social','desconocido_channel']
    channel=channel.reset_index()
    channel=channel.fillna(0)
    archivo_con_storage_condition_mark_cha=pd.merge(archivo_con_storage_condition_mark,channel,on='person',how='left')
    archivo_con_storage_condition_mark_cha=archivo_con_storage_condition_mark_cha.fillna(0)

    return archivo_con_storage_condition_mark_cha
    
    

In [192]:
data_train=agregarcolumnas(events_training,train_con_feature)
data_train.info()

1171886
19128
19128
19128
19128
19414
19414
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Data columns (total 48 columns):
person                       19414 non-null object
ad campaign hit              19414 non-null float64
brand listing                19414 non-null float64
checkout                     19414 non-null float64
conversion                   19414 non-null float64
generic listing              19414 non-null float64
lead                         19414 non-null float64
search engine hit            19414 non-null float64
searched products            19414 non-null float64
staticpage                   19414 non-null float64
viewed product               19414 non-null float64
visited site                 19414 non-null float64
mean_per_event               19414 non-null float64
most_frequent_event          19414 non-null object
count_most_frequent_event    19414 non-null int64
less_frequent_event          19414 non-null object
count_less_frequent_

In [193]:
data_test=agregarcolumnas(events_test,test_con_feature)
data_test.head() 

1169795
19122
19122
19122
19122
19415
19415


,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,Sony,iPad,iPhone,Direct,Email,Organic,Paid,Referral,Social,desconocido_channel
0,4886f805,0.00,0.00,1.00,0.00,1.00,0.00,1.00,1.00,0.00,...,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
1,0297fc1e,29.00,4.00,7.00,0.00,21.00,1.00,0.00,6.00,0.00,...,0.00,5.00,405.00,56.00,0.00,1.00,21.00,17.00,0.00,0.00
2,2d681dd8,1.00,5.00,1.00,0.00,1.00,0.00,2.00,1.00,0.00,...,0.00,0.00,6.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00
3,cccea85e,15.00,7.00,1.00,0.00,20.00,0.00,26.00,1.00,5.00,...,22.00,0.00,8.00,1.00,0.00,14.00,6.00,1.00,0.00,0.00
4,4c8a8b93,14.00,8.00,2.00,0.00,14.00,0.00,13.00,9.00,0.00,...,0.00,0.00,176.00,2.00,0.00,3.00,6.00,9.00,0.00,0.00


In [128]:
#agrega columnas test

In [194]:
data_train.to_csv("train_paso_3.csv", index = False)

In [195]:
data_test.to_csv("test_paso_3.csv", index = False)